# Réseaux convolutionnels pour le traitement de l'image 


## Partie A : Prise en main des CNN, introspection

Vincent Guigue

[directement inspiré de:]
Nicolas Baskiotis (nicolas.baskiotis@soronne-univeriste.fr) Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

Les objectifs de ce module sont :
* Prise en main des réseaux convolutionnels (CNN)
* Apprentissage d'un CNN
* Introspection d'un CNN

Nous travaillerons dans un premier temps avec les données MNIST puis avec le jeu de données CIFAR d'images de 10 classes.

In [2]:
# ! pip install tdqm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=7173c90b21c64fa43050822a20b50555459ed4bb05708ed6633ed6cdb52900f2
  Stored in directory: /Users/vguigue/Library/Caches/pip/wheels/86/cd/38/f96ed05dd8049e95d8fbeaa0587664eb001a1848979636b771
Successfully built tdqm


In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader,Dataset,random_split

import time
import os


### Prise en main du module sur un exemple jouet

In [2]:
x = torch.tensor([float(i) for i in range (25)]).view(5,5)
print(x)


tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24.]])


In [3]:

conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, bias=False) # 1 seul filtre, sans biais pour bien maitriser
# note 1: normalement, on laisse TOUJOURS le biais
# initialisation à la main pour tester (la procédure normale consiste à laisser l'init aléatoire)
poids = torch.tensor([[[[1., 2.],[1., 1.]]]])
conv.weight = nn.Parameter(poids)


In [4]:


print("out :", conv(x.unsqueeze(0)))
print("filtre : ", conv.weight)

# explication de la première sortie (= 13)
print("calcul à la main: ", (x[:2,:2] * poids).sum())

out : tensor([[[ 13.,  18.,  23.,  28.],
         [ 38.,  43.,  48.,  53.],
         [ 63.,  68.,  73.,  78.],
         [ 88.,  93.,  98., 103.]]], grad_fn=<SqueezeBackward1>)
filtre :  Parameter containing:
tensor([[[[1., 2.],
          [1., 1.]]]], requires_grad=True)
calcul à la main:  tensor(13.)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [9]:
# si on veut ajouter du padding => tout autour => une dimension de plus en sortie

conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, bias=False, padding=1, padding_mode='zeros') # 1 seul filtre, sans biais pour bien maitriser
# initialisation à la main pour tester (la procédure normale consiste à laisser l'init aléatoire)
poids = torch.tensor([[[[1., 2.],[1., 1.]]]])
conv.weight = nn.Parameter(poids)

print("out :", conv(x.unsqueeze(0))) # essayer d'enlever le unsqueeze + comprendre la logique

out : tensor([[[  0.,   1.,   3.,   5.,   7.,   4.],
         [  5.,  13.,  18.,  23.,  28.,  13.],
         [ 20.,  38.,  43.,  48.,  53.,  23.],
         [ 35.,  63.,  68.,  73.,  78.,  33.],
         [ 50.,  88.,  93.,  98., 103.,  43.],
         [ 40.,  62.,  65.,  68.,  71.,  24.]]], grad_fn=<SqueezeBackward1>)


In [10]:
# jouer avec les pas de déplacement de la fenêtre (stride = PAS)

conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, bias=False, stride = 2) # 1 seul filtre, sans biais pour bien maitriser
# initialisation à la main pour tester (la procédure normale consiste à laisser l'init aléatoire)
poids = torch.tensor([[[[1., 2.],[1., 1.]]]])
conv.weight = nn.Parameter(poids)

print("out :", conv(x.unsqueeze(0)))


out : tensor([[[13., 23.],
         [63., 73.]]], grad_fn=<SqueezeBackward1>)


In [11]:
# DILATATION - jouer avec les pas de déplacement de la fenêtre (dilation = filtre intermittent)

conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, bias=False, dilation = 2) # 1 seul filtre, sans biais pour bien maitriser
# initialisation à la main pour tester (la procédure normale consiste à laisser l'init aléatoire)
poids = torch.tensor([[[[1., 2.],[1., 1.]]]])
conv.weight = nn.Parameter(poids)

print("out :", conv(x.unsqueeze(0)))


out : tensor([[[26., 31., 36.],
         [51., 56., 61.],
         [76., 81., 86.]]], grad_fn=<SqueezeBackward1>)
